<h1>📘 Biofilter Queries — Quick Reference Guide</h1>

The Query class provides an interactive, notebook-friendly interface for exploring all database content using clean, Pythonic SQLAlchemy statements. It allows you to build and execute SQL queries, inspect models, and retrieve results as pandas DataFrames with minimal boilerplate.

Methods:
* ...

<h4>Models vs. Tables</h4>

In Biofilter3R, a model is a Python ORM class that represents a biological concept (such as a gene, variant, or disease) and defines its columns, relationships, and behavior inside the system. A table, on the other hand, is the physical structure stored in the database where the actual data lives. Every model maps to one table, but models provide additional structure—such as relationships, typing, and a user-friendly API—while tables provide efficient storage and indexing. In practice, you query models when working in Python (e.g., select(GeneMaster)), and you reference tables when executing raw SQL (SELECT * FROM gene_master). The distinction allows Biofilter3R to offer both a high-level, intuitive interface for exploration and a low-level interface for performance and debugging.

<h4>Query vs. Report</h4>
In Biofilter3R, the Query interface provides low-level, flexible access to the database, allowing users to explore 
tables and models interactively using SQLAlchemy expressions or raw SQL. It is designed for ad-hoc analysis, 
data inspection, and exploratory work inside notebooks. In contrast, a Report is a high-level, structured analysis module that encapsulates a complete workflow—from parameter validation to querying, post-processing, and formatted output. Reports are reusable, versioned, and intended for standardized scientific or operational tasks, whereas Queries are free-form, exploratory, and user-driven.

---------

### Start Biofilter3R

In [1]:
from biofilter import Biofilter

In [2]:
# Instance of Biofilter
bf = Biofilter()

[INFO] ════════════════════════════════════
[INFO] 🚀 Initializing Biofilter3R
[INFO]    • Version: 3.2.0
[INFO]    • Debug mode: False
[INFO]    • Config: /home/bioadmin/biofilter/.biofilter.toml
[INFO] ════════════════════════════════════
[INFO] 🔌 Database connection established
[INFO]    • Engine: postgresql+psycopg2
[INFO]    • Host:   localhost
[INFO]    • DB:     biofilter
[INFO]    • Time:   0.9 ms
[INFO] ════════════════════════════════════


--------

### 1. Access a model by name

In [3]:
bf.query.GeneMaster

biofilter.db.models.model_genes.GeneMaster

------

### 2. Query a model using simple filters
Executes an equality-based query on the chosen model using keyword arguments (e.g., get("GeneMaster", hgnc_id="5")). Automatically resolves the model name, builds the SQL query, and returns the results as a pandas DataFrame. Useful for quick lookups, filtering by one or more columns, and performing fast exploratory queries without writing full SQLAlchemy statements.

get() will:
* Resolve model name strings
* Accept only scalar equality filters
* Execute a simple SELECT * FROM model WHERE ...
* Return a clean DataFrame

query_model will not:
* Handle lists (IN (...))
* Handle ranges (>, <, BETWEEN)
* Handle LIKE, ILIKE, regex
* Perform joins between tables
* Return ORM objects

In [4]:
df = bf.query.get("GeneMaster", symbol="A1BG") # use string
df

,id,symbol,hgnc_status,omic_status_id,entity_id,chromosome,data_source_id,etl_package_id,locus_group_id,locus_type_id
0,1,A1BG,Approved,1,1,19,2,6,1,1


In [5]:
df = bf.query.get(
    bf.query.VariantSNP,  # use class object
    chromosome="1",
    position_38=175292543
)
df

,rs_id,chromosome,position_37,position_38,reference_allele,alternate_allele,data_source_id,etl_package_id
0,171,1,175261679,175292543,T,C,4,70


-------------

### 3. Query a model using complex statements

The model() method executes a SQLAlchemy statement (or model class) and always returns a pandas DataFrame.
It is designed for interactive exploration and quick data inspection in python scripts.

#### 3.1. Load all records from a model

Passing a model class directly is equivalent to running SELECT * FROM table.
This is the fastest way to preview the contents of a table.

In [4]:
df = bf.query.run_model(bf.query.GeneMaster)
df.head()

,id,symbol,hgnc_status,omic_status_id,entity_id,chromosome,data_source_id,etl_package_id,locus_group_id,locus_type_id
0,1,A1BG,Approved,1,1,19,2,6,1,1
1,2,A1BG-AS1,Approved,1,2,19,2,6,2,2
2,3,A1CF,Approved,1,3,10,2,6,1,1
3,4,A2M,Approved,1,4,12,2,6,1,1
4,5,A2M-AS1,Approved,1,5,12,2,6,2,2


#### 3.2 Limit the number of rows
Use select() when you want to control limits, filters, or joins explicitly.

In [5]:
select = bf.query.select # Select Method
genes = bf.query.GeneMaster # GeneMaster Model Class = Table

stmt = select(genes).limit(10)

df = bf.query.run_model(stmt)
df

,id,symbol,hgnc_status,omic_status_id,entity_id,chromosome,data_source_id,etl_package_id,locus_group_id,locus_type_id
0,1,A1BG,Approved,1,1,19,2,6,1,1
1,2,A1BG-AS1,Approved,1,2,19,2,6,2,2
2,3,A1CF,Approved,1,3,10,2,6,1,1
3,4,A2M,Approved,1,4,12,2,6,1,1
4,5,A2M-AS1,Approved,1,5,12,2,6,2,2
5,6,A2ML1,Approved,1,6,12,2,6,1,1
6,7,A2ML1-AS1,Approved,1,7,12,2,6,2,2
7,8,A2ML1-AS2,Approved,1,8,12,2,6,2,2
8,9,A3GALT2,Approved,1,9,1,2,6,1,1
9,10,A4GALT,Approved,1,10,22,2,6,1,1


#### 3.3. Filter using a WHERE clause
This example retrieves a specific gene using an equality-based filter.

In [7]:
# Used objects from step 2.2
stmt = (select(genes).where(genes.symbol == "A1BG"))
df = bf.query.run_model(stmt)
df

,id,symbol,hgnc_status,omic_status_id,entity_id,chromosome,data_source_id,etl_package_id,locus_group_id,locus_type_id
0,1,A1BG,Approved,1,1,19,2,6,1,1


#### 3.4. Query variants by chromosome and position
Multiple where() clauses can be chained to build precise queries.

In [8]:
VariantSNP = bf.query.VariantSNP
select = bf.query.select

stmt = (
    select(VariantSNP)
    .where(VariantSNP.chromosome == "1")
    .where(VariantSNP.position_38 == 53213657)
)

df = bf.query.run_model(stmt)
df

,rs_id,chromosome,position_37,position_38,reference_allele,alternate_allele,data_source_id,etl_package_id
0,672,1,53679329,53213657,G,A/C,4,70


#### 3.5. Query multiple records using IN
Use .in_() to retrieve multiple records in a single query.

In [9]:
# Used objects from step 2.4

positions = [53213657, 53214012, 53214555]

stmt = (
    select(VariantSNP)
    .where(VariantSNP.chromosome == "1")
    .where(VariantSNP.position_38.in_(positions))
)

df = bf.query.run_model(stmt)
df

,rs_id,chromosome,position_37,position_38,reference_allele,alternate_allele,data_source_id,etl_package_id
0,672,1,53679329,53213657,G,A/C,4,70
1,2525600908,1,53680227,53214555,G,T,4,70


#### 3.6. Select specific columns
Selecting explicit columns reduces memory usage and improves readability when you only need a subset of fields.

In [12]:
# Used objects from step 2.2
stmt = select(
    genes.id,
    genes.symbol,
    genes.hgnc_status,
    genes.entity_id
)

df = bf.query.run_model(stmt)
df

,id,symbol,hgnc_status,entity_id
0,1,A1BG,Approved,1
1,2,A1BG-AS1,Approved,2
2,3,A1CF,Approved,3
3,4,A2M,Approved,4
4,5,A2M-AS1,Approved,5
...,...,...,...,...
72524,72525,MVCD5,Gene from NCBI,88648
72525,72526,DEL2P16.3,Gene from NCBI,88650
72526,72527,PHN,Gene from NCBI,88653
72527,72528,LOC143923313,Gene from NCBI,88655


#### 3.7. Join two models
Joins allow you to combine information from related models using standard SQLAlchemy syntax.

In [34]:
GeneMaster = bf.query.GeneMaster
Alias = bf.query.EntityAlias  # melhor nome

stmt = (
    bf.query.select(
        GeneMaster.symbol.label("gene symbol"),
        # Alias.entity_id.label("entity_id"),
        Alias.xref_source.label("source"),
        Alias.alias_value.label("alias"),
        Alias.alias_type.label("type alias"),
        Alias.is_primary.label("principal alias"),
    )
    .select_from(GeneMaster)
    .join(Alias, Alias.entity_id == GeneMaster.entity_id)
     # .where(Alias.is_primary.is_(True)) # to filter only principal alias
    .limit(8)
)

df = bf.query.run_model(stmt)
df

,gene symbol,source,alias,type alias,principal alias
0,A1BG,HGNC,A1BG,symbol,True
1,A1BG,HGNC,HGNC:5,code,False
2,A1BG,ENSEMBL,ENSG00000121410,code,False
3,A1BG,ENTREZ,1,code,False
4,A1BG,UCSC,uc002qsd.5,code,False
5,A1BG,HGNC,alpha-1-B glycoprotein,synonym,False
6,A1BG-AS1,HGNC,A1BG-AS1,symbol,True
7,A1BG-AS1,HGNC,HGNC:37133,code,False


#### 3.8. Order results
Use order_by() to control sorting of query results.

In [18]:
stmt = (
    bf.query.select(bf.query.GeneMaster)
    .order_by(bf.query.GeneMaster.symbol)
    .limit(10)
)

df = bf.query.run_model(stmt)
df

,id,symbol,hgnc_status,omic_status_id,entity_id,chromosome,data_source_id,etl_package_id,locus_group_id,locus_type_id
0,57864,A-GAMMA3'E,Gene from NCBI,1,57864,11,1,9,9,3
1,12,A12M1,Entry Withdrawn,1,12,None,2,6,3,3
2,13,A12M2,Entry Withdrawn,1,13,None,2,6,3,3
3,14,A12M3,Entry Withdrawn,1,14,None,2,6,3,3
4,15,A12M4,Entry Withdrawn,1,15,None,2,6,3,3
5,1,A1BG,Approved,1,1,19,2,6,1,1
6,2,A1BG-AS1,Approved,1,2,19,2,6,2,2
7,3,A1CF,Approved,1,3,10,2,6,1,1
8,4,A2M,Approved,1,4,12,2,6,1,1
9,5,A2M-AS1,Approved,1,5,12,2,6,2,2


#### 3.9. Count records
Aggregate queries work the same way and return a DataFrame with the computed values.

In [22]:
stmt = bf.query.select(
    bf.query.func.count(bf.query.GeneMaster.id)
)

df = bf.query.run_model(stmt)
df

,value
0,72529


#### 3.10. Use model() as a safe exploratory tool
run_model() always returns a DataFrame and catches database errors internally, making it safe for exploratory analysis without breaking the notebook.


In [4]:
df = bf.query.run_model(
    bf.query.select(bf.query.DiseaseMaster).limit(5)
)
df

,id,disease_id,label,description,omic_status_id,entity_id,data_source_id,etl_package_id
0,1,MONDO:0000002,"obsolete 46,XX sex reversal",,2,114901,37,24
1,2,MONDO:0000003,obsolete 17-hydroxysteroid dehydrogenase defic...,,2,114904,37,24
2,3,MONDO:0000004,adrenocortical insufficiency,An endocrine or hormonal disorder that occurs ...,1,114907,37,24
3,4,MONDO:0000005,"alopecia, isolated",,1,114912,37,24
4,5,MONDO:0000006,obsolete alopecia-mental retardation syndrome,,2,114915,37,24


-------------

### 4. Select data using SQL code

The run_sql() method executes raw SQL statements directly against the Biofilter3R database and always returns the results as a pandas DataFrame.
It is intended for advanced users, debugging, validation, or situations where writing SQL is faster or more expressive than building a SQLAlchemy statement.

Unlike run_model(), run_sql() bypasses the ORM layer entirely.

#### When to use run_sql()

Use run_sql() when you need to:

* Quickly inspect a table using plain SQL
* Debug or validate ORM-generated queries
* Run database-specific SQL (CTEs, window functions, vendor-specific syntax)
* Explore indexes, counts, or performance-related questions
* Copy-paste queries from database consoles (psql, pgAdmin, etc.)

#### 4.1 Preview an entire table
This is the fastest way to inspect raw table contents without using ORM models.

In [3]:
df = bf.query.run_sql(
    "SELECT * FROM gene_masters LIMIT 10"
)
df

,id,symbol,hgnc_status,omic_status_id,entity_id,chromosome,data_source_id,etl_package_id,locus_group_id,locus_type_id
0,1,A1BG,Approved,1,1,19,2,6,1,1
1,2,A1BG-AS1,Approved,1,2,19,2,6,2,2
2,3,A1CF,Approved,1,3,10,2,6,1,1
3,4,A2M,Approved,1,4,12,2,6,1,1
4,5,A2M-AS1,Approved,1,5,12,2,6,2,2
5,6,A2ML1,Approved,1,6,12,2,6,1,1
6,7,A2ML1-AS1,Approved,1,7,12,2,6,2,2
7,8,A2ML1-AS2,Approved,1,8,12,2,6,2,2
8,9,A3GALT2,Approved,1,9,1,2,6,1,1
9,10,A4GALT,Approved,1,10,22,2,6,1,1


#### 4.2 Adding complexity with a more advanced query.
In this example, we retrieve all relationships between Genes and Diseases and enrich the results by including the genomic locations of the Genes mapped to genome assembly GRCh38.

In [16]:
sql = """
WITH groups AS (
  SELECT
    MAX(CASE WHEN name = 'Genes' THEN id END)     AS gene_group_id,
    MAX(CASE WHEN name = 'Diseases' THEN id END)  AS disease_group_id
  FROM entity_groups
),
gd_rels AS (
  SELECT
    er.*,

    CASE
      WHEN er.entity_1_group_id = g.gene_group_id THEN er.entity_1_id
      ELSE er.entity_2_id
    END AS gene_entity_id,

    CASE
      WHEN er.entity_1_group_id = g.disease_group_id THEN er.entity_1_id
      ELSE er.entity_2_id
    END AS disease_entity_id

  FROM entity_relationships er
  CROSS JOIN groups g
  WHERE
    (er.entity_1_group_id = g.gene_group_id AND er.entity_2_group_id = g.disease_group_id)
    OR
    (er.entity_1_group_id = g.disease_group_id AND er.entity_2_group_id = g.gene_group_id)
)

SELECT
  -- r.id AS relationship_id,

  -- Gene
  -- r.gene_entity_id,
  -- 'Gene' AS gene_group,
  ga.alias_value AS gene_symbol,

  -- Gene location (GRCh38)
  -- el.assembly_id,
  el.build,
  el.chromosome,
  el.start_pos,
  el.end_pos,
  -- el.strand,
  -- el.region_label,

  -- Relationship
  rt.code AS relationship_type,
  
  -- Disease
  -- r.disease_entity_id,
  -- 'Disease' AS disease_group,
  -- dm.disease_id AS disease_id,
  COALESCE(dm.label, da.alias_value) AS disease_name,

  -- Provenance
  -- ds.name AS data_source,
  ss.name AS source_system

FROM gd_rels r
JOIN entity_relationship_types rt
  ON rt.id = r.relationship_type_id

-- Gene primary alias
LEFT JOIN entity_aliases ga
  ON ga.entity_id = r.gene_entity_id
 AND (ga.is_primary = TRUE OR ga.alias_type = 'preferred')

-- Disease alias (usually MONDO code)
LEFT JOIN entity_aliases da
  ON da.entity_id = r.disease_entity_id
 AND (da.is_primary = TRUE OR da.alias_type = 'preferred')

-- Disease master (human-readable label)
LEFT JOIN disease_masters dm
  ON dm.entity_id = r.disease_entity_id

-- Gene location
LEFT JOIN entity_locations el
  ON el.entity_id = r.gene_entity_id
 AND el.build = 38

-- Provenance joins
LEFT JOIN etl_data_sources ds
  ON ds.id = r.data_source_id
LEFT JOIN etl_source_systems ss
  ON ss.id = ds.source_system_id

ORDER BY r.id
LIMIT 1000
"""
df = bf.query.run_sql(sql)
df.head()
df

,gene_symbol,build,chromosome,start_pos,end_pos,relationship_type,disease_name,source_system
0,RGS9,38,17,65100812,65227703,part_of,bradyopsia,CLINGEN
1,RHO,38,3,129528639,129535344,part_of,inherited retinal dystrophy,CLINGEN
2,GRK1,38,13,113667219,113737736,part_of,Oguchi disease,CLINGEN
3,RIPK1,38,6,3063824,3115187,part_of,immunodeficiency 57,CLINGEN
4,RIT1,38,1,155897808,155911408,part_of,Noonan syndrome,CLINGEN
